# Aula extra - Como analisar os dados de rentabilidade?

### Desafio:

* Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes. Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

* Essa aula será mais curta. 

### Passo a passo:

   **Passo 1** - Puxar as tabelas igual na aula 2.
   
   **Passo 2** - Escolher quais períodos de rentabilidade iremos analisar.
   
   **Passo 3** - Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

   **Passo 4** - Transformar as rentabilidades em números decimais, ao invés de string. 
   
   **Passo 5** - Juntar as tabelas, igual na aula 2. 
   
   **Passo 6** - Filtrar os ETFs alavancados. 
   
   **Passo 7** - Fazer os rankings de rentabilidade por período e soma-los. 
   
   **Passo 8** - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

# Passo 1: Puxar as tabelas igual na aula 2.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

time.sleep(5)

botao_100 = driver.find_element("xpath",
                               '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

driver.execute_script("arguments[0].click();", botao_100)

numero_paginas = driver.find_element("xpath", '//*[@id="totalPages"]')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

html_tabela = elemento.get_attribute('outerHTML')

lista_tabela_por_pagina = []

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
    
tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)


formulario_de_voltar_pagina = driver.find_element("xpath", '//*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys("1")
formulario_de_voltar_pagina.send_keys(u'\ue007')

botao_mudar_pra_performance = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')

driver.execute_script("arguments[0].click();", botao_mudar_pra_performance)

# a partir daqui é tudo igual

lista_tabela_por_pagina = []

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
    
tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)

driver.quit()
                                                               

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 4.95MB/s]


In [3]:
tabela_cadastro_etfs = tabela_cadastro_etfs.set_index('Ticker')
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index("Ticker")

# Passo 2: Escolher quais períodos de rentabilidade iremos analisar.

No nosso caso, iremos pegar as janelas de 1, 3 e 5 anos. 

In [4]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year', '3 Years', '5 Years']]

tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-9.01%,19.91%,11.05%
IVV,-9.02%,19.97%,11.06%
VOO,-9.02%,19.98%,11.07%
VTI,-10.30%,19.93%,10.25%
QQQ,-12.94%,21.03%,15.65%
...,...,...,...
CVLC,--,--,--
CVMC,--,--,--
CVIE,--,--,--


# Passo 3: Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

In [5]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.replace("--", pd.NA)

tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.dropna()

# Passo 4: Transformar as rentabilidades em números decimais

In [6]:
for coluna in tabela_rentabilidade_etfs.columns:
    
    tabela_rentabilidade_etfs[coluna] = tabela_rentabilidade_etfs[coluna].str.rstrip("%").astype(float)/100

# Passo 5: Juntar as tabelas, igual na aula 2.

In [7]:
base_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how = "inner")

base_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-0.0901,0.1991,0.1105
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-0.0902,0.1997,0.1106
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-0.0902,0.1998,0.1107
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-0.1030,0.1993,0.1025
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-0.1294,0.2103,0.1565
...,...,...,...,...,...,...,...,...
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0942,0.1026,-0.0212
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,0.0000,-0.3319,-0.2274
LTL,ProShares Ultra Communication Services,Leveraged Equity: U.S. Communication Services,ProShares,0.95%,$692.87K,-0.2630,0.1248,0.0136


# Passo 6: Filtrar os ETFs alavancados.

In [8]:
base_final = base_final[~base_final['Segment'].str.contains("Leveraged")]

base_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-0.0901,0.1991,0.1105
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-0.0902,0.1997,0.1106
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-0.0902,0.1998,0.1107
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-0.1030,0.1993,0.1025
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-0.1294,0.2103,0.1565
...,...,...,...,...,...,...,...,...
SDP,ProShares UltraShort Utilities,Inverse Equity: U.S. Utilities,ProShares,0.95%,$1.96M,0.0526,-0.2945,-0.2642
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0942,0.1026,-0.0212
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,0.0000,-0.3319,-0.2274


# Passo 7: Fazer os rankings de rentabilidade por período e soma-los. 

In [9]:
base_final['rank_1_anos'] = base_final['1 Year'].rank(ascending = False)
base_final['rank_3_anos'] = base_final['3 Years'].rank(ascending = False)
base_final['rank_5_anos'] = base_final['5 Years'].rank(ascending = False)
base_final['rank_final'] = (base_final['rank_1_anos'] + 
                                  base_final['rank_3_anos'] + 
                                  base_final['rank_5_anos'])

base_final

C:\Users\Felipe Belmont\AppData\Local\Temp\ipykernel_28564\3578762214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['rank_1_anos'] = base_final['1 Year'].rank(ascending = False)
C:\Users\Felipe Belmont\AppData\Local\Temp\ipykernel_28564\3578762214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['rank_3_anos'] = base_final['3 Years'].rank(ascending = False)
C:\Users\Felipe Belmont\AppData\Local\Temp\ipykernel_28564\3578762214.py:3: SettingWithCopyWarning: 
A value is trying to

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,rank_1_anos,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$374.42B,-0.0901,0.1991,0.1105,920.5,434.0,101.0,1455.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$306.66B,-0.0902,0.1997,0.1106,923.5,423.5,99.5,1446.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$287.12B,-0.0902,0.1998,0.1107,923.5,421.5,97.5,1442.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$282.79B,-0.1030,0.1993,0.1025,1022.0,431.5,151.5,1605.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.68B,-0.1294,0.2103,0.1565,1170.5,340.0,14.0,1524.5
...,...,...,...,...,...,...,...,...,...,...,...,...
SDP,ProShares UltraShort Utilities,Inverse Equity: U.S. Utilities,ProShares,0.95%,$1.96M,0.0526,-0.2945,-0.2642,100.0,1444.0,1456.0,3000.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,-0.0942,0.1026,-0.0212,950.0,977.0,1299.0,3226.0
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,0.0000,-0.3319,-0.2274,256.5,1449.0,1446.0,3151.5


# Passo 8: Gran finale - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

In [10]:
base_final = base_final.sort_values(by = "rank_final")

base_final.head(20)

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,rank_1_anos,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,,,
UGA,United States Gasoline Fund LP,Commodities: Energy Gasoline,Marygold,0.93%,$74.99M,0.0931,0.7312,0.1455,59.0,2.0,20.0,81.0
ITB,iShares U.S. Home Construction ETF,Equity: U.S. Housing,Blackrock,0.39%,$1.64B,0.1548,0.3995,0.1200,20.0,37.0,50.0,107.0
AIRR,First Trust RBA American Industrial Renaissanc...,Equity: U.S. Industrials,First Trust,0.70%,$316.07M,0.0936,0.3608,0.1244,58.0,59.0,42.0,159.0
XLE,Energy Select Sector SPDR Fund,Equity: U.S. Energy,State Street Global Advisors,0.10%,$40.36B,0.1489,0.4855,0.1016,22.0,20.0,156.5,198.5
SGG,iPath Series B Bloomberg Sugar Subindex Total ...,Commodities: Agriculture Sugar,Barclays Capital Inc.,0.45%,$41.42M,0.2770,0.3192,0.1078,7.0,82.0,124.0,213.0
GRID,First Trust NASDAQ Clean Edge Smart Grid Infra...,Equity: Global Infrastructure,First Trust,0.58%,$719.10M,0.0181,0.3512,0.1558,156.0,65.0,16.0,237.0
IEO,iShares U.S. Oil & Gas Exploration & Productio...,Equity: U.S. Oil & Gas Exploration & Production,Blackrock,0.39%,$764.20M,0.0667,0.6024,0.1005,78.0,6.0,164.5,248.5
FENY,Fidelity MSCI Energy Index ETF,Equity: U.S. Energy,Fidelity,0.08%,$1.59B,0.1340,0.5000,0.0913,26.0,19.0,235.0,280.0
VDE,Vanguard Energy ETF,Equity: U.S. Energy,Vanguard,0.10%,$8.03B,0.1304,0.5050,0.0916,29.0,18.0,233.0,280.0
